In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
# Defining the device to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#hyperparams
epochs = 2
batchsize = 100
lr = 0.0001
inputsize = 28*28
hid_layer1_sz = 32
hid_layer2_sz = 32
hid_layer3_sz = 16
class_size = 10

In [3]:

#loading database
db = pd.read_csv('../../datasets/TMNIST_Data.csv')
features = db.iloc[:, 2:]
labels = db.labels

ftmean = np.nanmean(features) / 255
ftstd = features.values.std() / 255

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((ftmean,), (ftstd,))
])

In [5]:
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.features.iloc[idx].values.reshape(28, 28).astype('float32')
        label = self.labels.iloc[idx] if isinstance(self.labels, pd.Series) else self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [6]:
# SPLITTING OUR DATA INTO TRAINING AND TESTING:
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, train_size=0.8)

# WE INITIALIZE OUR TRAINING AND TESTING DATASETS:
train_db = MNISTDataset(xtrain,ytrain,transform)
test_db = MNISTDataset(xtest,ytest,transform)

# THIS IS THE DATASET/DATALOADER THAT WE USE FOR TRAINING
TRAIN_DL = torch.utils.data.DataLoader(train_db, batch_size=batchsize, shuffle=True)
TEST_DL = torch.utils.data.DataLoader(test_db, batch_size=batchsize, shuffle=False)

In [7]:

# Creating our model architecture
class NeuralNetwork(nn.Module):
    def __init__(self,inputsize, hidden1_sz, hidden2_sz, hidden3_sz, class_sz):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(inputsize,hidden1_sz)
        self.activation = nn.ReLU()
        self.layer2 = nn.Linear(hidden1_sz, hidden2_sz)
        self.layer3 = nn.Linear(hidden2_sz, hidden3_sz)
        self.layer4 = nn.Linear(hidden3_sz, class_sz)

    def forward(self, x):
        output = self.layer1(x)
        output = self.activation(output)
        output = self.layer2(output)
        output = self.activation(output)
        output = self.layer3(output)
        output = self.activation(output)
        output = self.layer4(output)
        output = self.activation(output)
        return output

In [8]:
# Initializing our model
model = NeuralNetwork(inputsize,hid_layer1_sz,hid_layer2_sz,hid_layer3_sz,class_size).to(device)

# Defining our loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [9]:
n_total_steps = len(TRAIN_DL)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(TRAIN_DL):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')


Epoch [1/2], Step [100/240], loss: 2.1248
Epoch [1/2], Step [200/240], loss: 1.5164
Epoch [2/2], Step [100/240], loss: 0.9312
Epoch [2/2], Step [200/240], loss: 0.8408


In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = len(TEST_DL.dataset)
    for images, labels in TEST_DL:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
    
        outputs = model(images)
        a, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()
    
    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 5980 test images: 78.61204013377926 %
